In [10]:
import geopandas as gpd
import folium
import pandas as pd

In [11]:
%run "../scripts/ETL.py"

In [19]:
%run "External data EDA.ipynb"

In [12]:
final_join.head()
num_transactions_by_postcode = final_join.groupBy('postcode').count()
num_transactions_by_postcode.head()

Row(postcode='7252', count=2682)

In [13]:
num_transactions_by_postcode.head()
num_transactions_by_postcode.count()

3166

In [14]:
postcodes_data1 = spark.read.option("header", True).csv('../data/postcodes.csv')
postcodes_data2 = postcodes_data1.withColumnRenamed('Postcode', 'postcode')
postcodes_data = postcodes_data2.dropDuplicates(['postcode'])

In [15]:
transactions_location = num_transactions_by_postcode.join(postcodes_data,['postcode'], 'inner')
transactions_location.head()


Row(postcode='2136', count=1026, Suburb='Burwood Heights', State='NSW', Lat='-33.890', Lon='151.100')

In [16]:
transactions_location.count()

3136

As we can see above - 30 postcodes in the transactions dataset were not found in the postcodes information dataset and hence their lattitude and longitude values were not retrieved

In [31]:
boundaries.head(5)
len(boundaries)

2454

In [30]:
## The following was modified from MAST30034 Tutorial 2
# read in shape file

transactions_location_pdf = transactions_location.toPandas()

# join transaction location df with shape file
gdf = gpd.GeoDataFrame(
    pd.merge(transactions_location_pdf, boundaries, left_on='Suburb', right_on='SA2_NAME21')
)

len(gdf)




493

In [ ]:
aus_coords = [-25.2744, 133.7751]
m = folium.Map(aus_coords, tiles='OpenStreetMap', zoom_start=4.5)

for index, row in transactions_location_pdf.iterrows():
    if row['count'] >= 5000:
        marker_color = 'darkred'
        fill_color = 'darkred'
    elif row['count'] >= 1000:
        marker_color = 'red'
        fill_color = 'red'
    elif row['count'] >= 500:
        marker_color = 'lightred'
        fill_color = 'lightred'
    elif row['count'] >= 100:
        marker_color = 'orange'
        fill_color = 'orange'
    elif row['count'] <= 50 :
        marker_color = 'yellow'
        fill_color = 'yellow'
    else:
        marker_color='darkpurple'
        fill_color = 'darkpurple'
        
    folium.Circle(
          location=[row['Lat'], row['Lon']],
          popup= 'Number of transactions: ' +str(row['count']),
          tooltip=row['Suburb'],
          radius=row['count'],
          color=marker_color,
          fill=True,
          fill_color=fill_color,
       ).add_to(m)
m